<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Intro To SQL: Connecting to Databases

_Authors: Dave Yerrington (SF), Fancesco Mosconi_

---

### Learning Objectives
*After this lesson, you will be able to:*
- Connect to a local or remote database using the command line, Python or Pandas
- Connect to a local or remote database using SQLite Manager (for SQLite) or Postico(for POSTGRES)
- Perform queries using SELECT
- Perform simple aggregations COUNT, MAX/MIN/SUM

### Lesson Guide
- [Connecting to a local database](#connecting-to-localdb)
    - [SQLite](#sqlite)
    - [Interacting with SQLite](#interacting-sqlite)
    - [Common SQL command patterns](#command-patterns)
    - [SQLite command line utility](#sqlite-command-line)
    - [Creating tables and adding columns](#sqlite-tables-columns)
    - [Adding data](#adding-data)
    - [Updating records](#updating-records)
    - [Removing records](#removing-records)
- [Connecting to a remote database](#remote-database)
    - [Postgresql](#postgresql)
- [Comparison of SQL commands](#comparison-commands)
- [SQLite from Python](#python-sqlite)
    - [sqlite3 package](#sqlite3)
    - [sqlite3: adding data](#sqlite3-adding)
    - [Adding data from a csv file](#adding-csv)
- [Pandas connector](#pandas-connector)
    - [Writing data into a database](#pandas-writing)
    - [Reading data from a database](#pandas-reading)
- [SQL syntax](#sql-syntax)
    - [Aggregations](#aggregations)
- [Independent practice](#ind-practice)

<a name="connecting-to-localdb"></a>
## Connecting to a Local Database

---

A database can be local or remote, it can span a single machine or it can be distributed with replicated data over several machines. The latter configuration is called _sharding_.

Let's start by connecting to a local _sqlite_ database.

<a id='sqlite'></a>

### SQLite

**[SQLite](https://sqlite.org/)** is a database software package built on the Structured Query Language [(SQL)](https://en.wikipedia.org/wiki/SQL).  It is similar to other SQL databases, such as [PostgreSQL](http://www.postgresql.org/), [MySQL](https://www.mysql.com/), Oracle, and Microsoft SQL Server, except that it is *file-based*, rather than *server-based*.  This makes it easy to setup and use for small projects, but less suitable for production environments.  Once you are familiar with sqlite, the same ideas and similar syntax can be applied to other SQL databases.

SQLite v3 is bundled with most python distributions (including our Anaconda distribution).  There are two options we recommend for browsing SQLite3 databases:

- [SQLite Browser](http://sqlitebrowser.org/), a free cross-platform solution
- [SQLite Manager](https://addons.mozilla.org/en-US/firefox/addon/sqlite-manager/?src), a Firefox add-on for viewing SQLite database files via a simple GUI.

<a id='interacting-sqlite'></a>

### Interacting with SQLite

There are multiple ways of interacting with an SQLite database, including:

1. SQLite Command Line Utility
2. python `sqlite3` package
3. `pandas` SQL Interface
4. High-level ORMs (e.g. sqlalchemy, django ORM, etc.)

Let's start with method 1. All of these methods provide some form of wrapper, or set of convenience functions, for interacting with SQLite.  Behind the scenes, the Structured Query Language (SQL) itself defines the interface to the database software.  This underlying SQL syntax will be visible to a greater or lesser degree depending upon the method that is chosen.

<a id='command-patterns'></a>

### Common SQL command Patterns

The SQL command set has a rich syntax with numerous options, but most of the commonly used commands follow a few simple patterns.  A basic familiarity of these patterns is helpful when working in SQL:

    CREATE TABLE ...
    ALTER TABLE ... ADD COLUMN ...
    INSERT INTO ... VALUES ...
    UPDATE ... SET ... WHERE ...
    SELECT ... FROM ... WHERE ...
    SELECT ... FROM ... JOIN ... ON ...
    DELETE FROM ... WHERE ...

<a id='sqlite-command-line'></a>

### SQLite Command Line Utility

The first method we'll explore is connecting to SQLite via the built-in [command line utility](https://www.sqlite.org/sqlite.html).  

> Note: the commands in this section should be executed within your normal terminal shell.

To start a new session of the interpreter, simply open your terminal and type `sqlite3`, followed by the name of the database file.  If the file does not yet exist, sqlite will create it.

    $ sqlite3 test1.sqlite

    SQLite version 3.7.12 2012-04-03 19:43:07
    Enter ".help" for instructions
    Enter SQL statements terminated with a ";"
    sqlite>

Notice that your terminal prompt changes to `sqlite>`, indicating that you are now entering commands into the sqlite command line utility.  Take a quick look at the help command:

    sqlite> .help
Display the current databases - you should see the new file `test1.db`:

    sqlite> .databases

<a id='sqlite-tables-columns'></a>

### Creating tables and adding columns

Create an table called `table1` with a single column `field1` containing an INTEGER PRIMARY KEY:

    sqlite> CREATE TABLE table1 (field1 INTEGER PRIMARY KEY);

Add a few more columns to `table1`:

    sqlite> ALTER TABLE table1 ADD COLUMN field2 VARCHAR(16);
    sqlite> ALTER TABLE table1 ADD COLUMN field3 REAL;
    sqlite> ALTER TABLE table1 ADD COLUMN field4 TEXT;

Notice the different field types in the ALTER TABLE commands.  SQLite supports several different [field types](https://www.sqlite.org/datatype3.html), including INTEGERS, variable length VARCHAR character fields (with a max length), TEXT fields, and 'REALS', which are used to store floating point numbers.

Verify that the table was created:

    sqlite> .tables

You can check the `schema` of the table using `.schema`, which shows the commands that would be needed to create the database tables from scratch.  

    sqlite> .schema

Notice that in this case, our `table1` could have been created with a single command, rather than adding each column separately.

<a id='adding-data'></a>

### Adding data

Let's add some data:

    sqlite> INSERT INTO table1 VALUES (1, 'Henry James', 42, '75 Mission Street, San Francisco, CA');
    sqlite> INSERT INTO table1 VALUES (2, 'Carol James', 40, '75 Mission Street, San Francisco, CA');
    sqlite> INSERT INTO table1 VALUES (3, 'Jesse James', 12, '75 Mission Street, San Francisco, CA');

Notice that the first column has unique values - this is a requirement for the PRIMARY KEY column.  If we try to add a record using an existing PK value we'll get an error:

    sqlite> INSERT INTO table1 VALUES (3, 'Julie James', 10, '75 Mission Street, San Francisco, CA');
    Error: PRIMARY KEY must be unique

Fortunately, SQLite has some built in functionality to auto-increment the PK value - just set the value of the PK field to NULL when doing the INSERT and it will automatically be set to a valid value.

    sqlite> INSERT INTO table1 VALUES (NULL, 'Julie James', 10, '75 Mission Street, San Francisco, CA');

Now that we have some data, take a look at the database using the **SQLite Browser**.

1. Open App
2. Open SQLite3 File "Open Database" from file menu
3. Click on "Browse Data" from tabbed UI

Notice that the value in `field1` for the Julie James record has been automatically set to 4.

### Updating records

Suppose we need to update an existing record with new data - e.g. maybe Julie James is only 9.  For this we use the UPDATE command:

    sqlite> UPDATE table1 SET field3=9 WHERE field1=4;

<a id='removing-records'></a>
### Removing Records

To remove records use the DELETE command:

    sqlite> DELETE FROM table1 WHERE field2 like '%Jesse%';

Use SQLite-Manager to verify that the Jesse James record has been removed.  To exit the sqlite interpreter type `.exit`.

    sqlite>  .exit

<a name="remote_database"></a>
## Connecting to a Remote Database

---

> You will need to update this with appropriate AWS configuration settings!

<a id='postgresql'></a>
### Postgresql

**[Postgresql](http://www.postgresql.org/)** is a very powerful SQL based relational database.

#### PostgreSQL syntax

GA provides a PostgreSQL database instance at the following address:

You can connect to it using:

> ### OSX Users
> For OSX, we install postgress via the PostgresApp package from [postgresapp.com](http://postgresapp.com/)
> Also, for easy access, add this alias to your .bash_profile:
>
> **Edit the file:** <br>
> `nano ~/.bash_profile`
>
> **Add this line:** <br>
> `alias psql=/Applications/Postgres.app/Contents/Versions/9.5/bin/psql`
>
> **Reload your profile:** <br>
> `source ~/.bash_profile`

    psql -h dsi.c20gkj5cvu3l.us-east-1.rds.amazonaws.com -p 5432 -U dsi_student titanic
    password: gastudents

PosgreSQL accepts the same syntax as sqlite, with exception of a few system commands. Here are the most common:

- `\q`: Quit/Exit
- `\c __database__`: Connect to a database
- `\d __table__`: Show table definition including triggers
- `\dt *.*`: List tables from all schemas (if `*.*` is omitted will only show SEARCH_PATH ones)
- `\l`: List databases
- `\dn`: List schemas
- `\df`: List functions
- `\dv`: List views
- `\dt`: List tables
- `\df+ __function` : Show function SQL code.
- `\x`: Pretty-format query results instead of the not-so-useful ASCII tables


Suggestions:
- if run with `-E` flag, it will describe the underlaying queries of the `\` commands (cool for learning!).
- Most `\d` commands support additional param of `__schema__.name__` and accept wildcards like `*.*`

<a id='comparison-commands'></a>
## Comparison of SQL commands

---


|Task|MySQL|PostgreSQL|SQLite|
|---|---|---|---|
|Connect to a database|mysql &lt;dbname&gt;| psql &lt;dbname&gt;|sqlite3 &lt;filename&gt;|
|Client help|help contents|\?|.help|
|SQL help|help contents|\h|n/a|
|List databases|SHOW DATABASES;|\l|.databases|
|Change database|USE &lt;dbname&gt;|\c <dbname&gt;|n/a|
|List tables|SHOW TABLES;|\dt|.tables|
|Show table|info DESCRIBE &lt;tablename&gt;|\d &lt;tablename&gt;|.schema &lt;tablename&gt;|
|Load data|LOAD DATA INFILE &lt;file&gt; |\i &lt;file&gt;|.import &lt;file&gt; &lt;table&gt;|
|Export data|SELECT ... INTO OUTFILE &lt;file&gt;|\o &lt;file&gt;|.dump &lt;table&gt;|
|Exit the client|quit (or exit)| \q|.exit|

<a name="intermission"></a>

## Intermission

---

We have seen how to connect to a local sqlite database and to a remote postgresql database.

**Check:** What SQL operations have we learned so far?

**Check:** What different commands have we learned for SQLite and PostgreSQL?


<a name="python-sqlite"></a>
## Interacting with SQLite from Python

---

<a id='sqlite3'></a>
### The `sqlite3` package

The command line utility can be useful for basic SQL tasks, but since we're using python it is often easier to access sqlite with python code directly. We can use the python [`sqlite3`](https://docs.python.org/2.7/library/sqlite3.html) package for just this purpose.

**Below we open a connection to an SQLite database file using python.  As before, if the file does not already exist it will automatically be created.**

In [25]:
import sqlite3
sqlite_db = './datasets/test_db.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

**The syntax to create a table is similar to the console, only now we use the `execute` method of the cursor object `c` that we just created:**

In [27]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')

# Save (commit) the changes
# conn.commit()

OperationalError: table houses already exists

With the database saved the table should now be viewable using SQLite Manager.

<a id='sqlite3-adding'></a>
### sqlite3: adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.

See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

In [28]:
last_sale = (None, 4000, 5, 22, 619000)
c.execute('INSERT INTO houses VALUES (?,?,?,?,?)', last_sale)

# Remember to commit the changes
conn.commit()

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key. 

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

In [29]:
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

c.executemany('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', recent_sales)

conn.commit()

Once again, use SQLite Browser to verify the database contents. 

![SQLite Manager](https://snag.gy/xc8W5M.jpg)

<a id='adding-from-csv'></a>

### Adding data from a csv file

Next let's load our housing.csv data into an array, and then `INSERT` those records into the database.  In this example we'll use the numpy `genfromtxt` function to read the file and parse the contents. 

In [36]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists
data = (genfromtxt('datasets/housing-data.csv', dtype='i8', 
                    delimiter=',', skip_header=1)).tolist()

# append a None value to beginning of each sub-list
for d in data:
    d.insert(0, None)

In [37]:
data[0:3]

[[None, 2104, 3, 70, 399900],
 [None, 1600, 3, 28, 329900],
 [None, 2400, 3, 44, 369000]]

In [38]:
# loop through data, running an INSERT on each record (i.e. sublist)
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

> **Remember:** all elements in a numpy array must be the same data type, so if we want to 'add a None' to each row, we need to work around this.  Lists can contain mixed types, so this is one approach.

Still, in this case the value we're adding is the same for all records, so we could have simply used a 'None' in the INSERT statement directly.

**How do you delete data?**

In [39]:
# similar syntax as before
results = c.execute("SELECT * FROM houses WHERE bdrms = 4")

# here results is a cursor object - use fetchall() to extract a list
results.fetchall()

[(2, 2390, 4, 34, 319000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (15, 1940, 4, 7, 239999),
 (20, 2300, 4, 77, 449900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (28, 1962, 4, 53, 259900),
 (37, 2040, 4, 75, 314900),
 (39, 1811, 4, 24, 285900),
 (42, 2132, 4, 28, 345000),
 (43, 4215, 4, 66, 549000),
 (44, 2162, 4, 43, 287000),
 (47, 2567, 4, 57, 314000),
 (50, 1852, 4, 64, 299900)]

<a id='pandas-connector'></a>

## Pandas connector

---

While databases provide many analytical capabilities, it's often useful to pull the data back into Python for more flexible programming. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales to present a report or dashboard, this operation is likely not changing and operating on a large dataset. This can run very efficiently in a database rather than by connecting to it with Python.

However, if we want to investigate login or sales data further and ask more interactive questions, then Python would be more practical.

In [33]:
import pandas as pd

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

SQLite creates portable SQL databases saved in a single file. These databases are stored in a very efficient manner and allow fast querying, making them ideal for small databases or databases that need to be moved across machines.

<a id='pandas-writing'></a>

### Writing data into a database

Data in Pandas can be loaded into a relational database. For the most part, Pandas can use column information to infer the schema for the table it creates. For the next demo we will use the Rossmann stores dataset.

In [42]:
data = pd.read_csv('./datasets/housing-data.csv', low_memory=False)  # lower_memory = gets rid of ambigious warning.. nothing to see here
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


Data is moved to the database through the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail
    
> ### BTW, what was with the `conn` object?  
> We defined in a previous example:
> ```python
import sqlite3
sqlite_db = './datasets/test_db.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()
```

In [43]:
data.to_sql('houses_pandas',             # Name of the table
            con=conn,                    # The handle to the file that is setup
            if_exists='replace',         # Overwrite, append, or fail
            index=False)                 # Add index as column

<a id='pandas-reading'></a>

### Reading data from a database
_Result as DataFrame_

If we already have data in our database, we can use Pandas to query it. Querying is done through the `read_sql` command in the `sql` module.

In [49]:
df = pd.read_sql('SELECT * FROM houses_pandas LIMIT 10', con=conn)
df

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900
5,1985,4,61,299900
6,1534,3,12,314900
7,1427,3,57,198999
8,1380,3,14,212000
9,1494,3,15,242500


<a name="sql-syntax"></a>

## SQL syntax

---

### SQL Operators

> Note: Each of the following can be demoed in pandas using the data we've setup above. A demo and check are included for each, but we can try a few of these things out before we .

#### SELECT

Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** What do `sqft`, `bdrms` and `price` refer to?  What will return?
>
```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE

`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is some filter applied to the rows, where rows that match the condition will be in the output.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for when houses older than 60 years.

In [55]:
sql_query = """
SELECT * 
FROM houses_pandas
"""

df = pd.read_sql(sql_query, con=conn)
df.groupby("bdrms").size()

bdrms
1     1
2     6
3    25
4    14
5     1
dtype: int64

### Aggregations

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate funtions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

In [62]:
sql_query = """
SELECT * FROM houses_pandas
"""

pd.read_sql(sql_query, con=conn).describe().T

,count,mean,std,min,25%,50%,75%,max
sqft,47.0,2000.680851,794.702354,852.0,1432.0,1888.0,2269.0,4478.0
bdrms,47.0,3.170213,0.760982,1.0,3.0,3.0,4.0,5.0
age,47.0,42.744681,22.873440,5.0,24.5,44.0,61.5,79.0
price,47.0,340412.659574,125039.899586,169900.0,249900.0,299900.0,384450.0,699900.0


<a name="ind-practice"></a>

## Independent practice: querying a database

---

Practice querying the SQLite database we've created using any of the methods you've learnt so far:

- console connection
- python `sqlite3` package
- pandas
- [SQLite Browser](http://www.sqlitebrowser.org/) or _firefox browser extension [SQLite Manager](https://addons.mozilla.org/en-US/firefox/addon/sqlite-manager/?src)_

Practice querying the PostgreSQL database you can find at [add url here](http://) using:
- console connection
- python `sqlite3` package
- pandas
- [Postico](https://eggerapps.at/postico/)


### What's the average price per room for 1 bedroom apartments?

In [96]:
sql_query = """
SELECT * FROM houses_pandas
WHERE bdrms = 1
"""

pd.read_sql(sql_query, con=conn)

,sqft,bdrms,age,price
0,1000,1,5,169900


### How old is the oldest 3 bedrooms apartment?

In [82]:
sql_query = """
SELECT MAX(age) FROM houses_pandas
WHERE bdrms = 3
"""

pd.read_sql(sql_query, con=conn)


,MAX(age)
0,78


### How old is the youngest apartment?

In [83]:
sql_query = """
SELECT MIN(age) FROM houses_pandas
"""

pd.read_sql(sql_query, con=conn)


,MIN(age)
0,5


### What's the average age for the whole dataset?

In [85]:
sql_query = """
SELECT AVG(age) FROM houses_pandas
WHERE bdrms = 3
"""

pd.read_sql(sql_query, con=conn)


,AVG(age)
0,38.36


### What's the average age for each bedroom size?

In [101]:
sql_query = """
SELECT AVG(age) FROM houses_pandas
WHERE bdrms = %d
"""

for bdrm in range(1, 6):
    print "\nAVG number of \"%d\" bdrm(s): " % bdrm, \
          pd.read_sql(sql_query % bdrm, con=conn), "\n", \
          sql_query % bdrm



AVG number of "1" bdrm(s):     AVG(age)
0       5.0 

SELECT AVG(age) FROM houses_pandas
WHERE bdrms = 1


AVG number of "2" bdrm(s):      AVG(age)
0  56.666667 

SELECT AVG(age) FROM houses_pandas
WHERE bdrms = 2


AVG number of "3" bdrm(s):     AVG(age)
0     38.36 

SELECT AVG(age) FROM houses_pandas
WHERE bdrms = 3


AVG number of "4" bdrm(s):      AVG(age)
0  46.857143 

SELECT AVG(age) FROM houses_pandas
WHERE bdrms = 4


AVG number of "5" bdrm(s):     AVG(age)
0      49.0 

SELECT AVG(age) FROM houses_pandas
WHERE bdrms = 5



**If you finish, try completing the first sections of [SQL zoo](http://www.sqlzoo.net).**

<a id='additional-resources'></a>

## Additional resources 

---

- [sqlite3 home](http://www.sqlite.org)  
- [SQLite - Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL zoo](http://www.sqlzoo.net)